In [44]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np

In [45]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:20.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:42:20.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:35.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:48.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:44:27.  Portanto não foi baixado novamente.
vrapeelo

In [46]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]

In [47]:
monitoramentodados  = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))
io_vrapeel = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[['DthEnvio','IdeUsinaOutorga','DatPrevistaAprovacaoIII','DatRealizacaoIII']].drop_duplicates(inplace=True)

In [48]:
rename_cols = {'DthEnvio' : 'Data_Envio',
'NomUsina' : 'usina_nome',
'DatRealizacaoII' : 'canteiroReal',
'DatPrevistaAprovacaoIII' : 'prev_IO',
'DatRealizacaoIII' : 'IO_real',
'DatRealizacaoIX' : 'DesvRio_real',
'DatRealizacaoXI' : 'Ench_Real',
'DatRealizacaoXIII' : 'IOTrans_Real',
'DatRealizacaoXIV' : 'Conc_Trans_REAL',
'DatRealizacaoXII' : 'Comiss_Real',
'DatRealizacaoVII' : 'ME_Real_conc_eol',
'DatRealizacaoVIII' : 'Conc_Combust_Real',}
cols = ['IdeUsinaOutorga','DthEnvio','CodCeg','NomUsina','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV',]
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [49]:
vcronograma.Data_Envio

AttributeError: 'DataFrame' object has no attribute 'Data_Envio'

In [ ]:
rename_cols = {
    'DatPrevistaComercial' : 'DatPrevistaComercial1'
}
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','SigTipoGeracao','DscObservacao','VlrPotenciaUnitaria','NumOperacaoUg','DatPrevistaComercial','DatPrevistaRotor','DatPrevistaMaximaRotor','DatRealizacaoRotor','DatPrevistaTeste','DatPrevistaMaximaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatPrevistaMaximaComercial','DatRealizacaoComercial','SigTipoGeracao','NomUsina']
ugrapeel  = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [ ]:
rename_cols = {
'DthEnvio' : 'Data_Envio',
'SigTipoGeracao' : 'TipoGeracao',
'NomUsina' : 'usina_nome',
'NumOperacaoUg' : 'Num_UG',
'VlrPotenciaUnitaria' : 'Pot_UG',
'DatRealizacaoRotor' : 'UG_decidarotor_real',
'DatPrevistaTeste' : 'UG_op_test_previsão',
'DatRealizacaoTeste' : 'UG_op_test_real',
'DatPrevistaComercial' : 'UG_op_com_previsão',
'DatRealizacaoComercial' : 'UG_op_com_real',
}

cols = ['DthEnvio','IdeUsinaOutorga','SigTipoGeracao','NomUsina','NumOperacaoUg','VlrPotenciaUnitaria','DatRealizacaoRotor','DatPrevistaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatRealizacaoComercial','DscObservacao','DthEnvio','IdeUsinaOutorga','NumOperacaoUg']
inputug = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [ ]:
cols = ['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','MdaPotenciaLiberadaTeste','MdaPotenciaLiberadaComercial','NumDespachoTeste','numDespachoComercial','IdcMonitorada','DatDescidaoutorgadaRotor','DatRealizacaoSFGRotor','DatInicioOpTesteOutorgada','DatLiberacaoSFGTeste','DatUGInicioOpComerOutorgado','DatLiberOpComerRealizado','DatPrevisaoSFGComercial']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].drop_duplicates()

In [ ]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','NomSituacaoI','PctEnergiaContratoAssinadoI','DatConclusaoI','NomSituacaoII','PctEnergiaContratoAssinadoII','DatConclusaoII','NomSituacaoIII','PctEnergiaContratoAssinadoIII','DatConclusaoIII','DscObservacao']
rename_cols = {'NomSituacaoI' : 'SituacaoACR',
'PctEnergiaContratoAssinadoI' : 'PctEnergiaContratoAssinadoACR',
'DatConclusaoI' : 'DatConclusaoACR',
'NomSituacaoII' : 'SituacaoCCESI',
'PctEnergiaContratoAssinadoII' : 'PctEnergiaContratoAssinadoCCESI',
'DatConclusaoII' : 'DatConclusaoCCESI',
'NomSituacaoIII' : 'SituacaoACL',
'PctEnergiaContratoAssinadoIII' : 'PctEnergiaContratoAssinadoACL',
'DatConclusaoIII' : 'DatConclusaoACL',
'DscObservacao' : 'DscObservacaomercado'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [ ]:
cols= ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','DatSolicitacaoI','DatPrevistaEmissaoI','DatEmissaoI','DatValidadeI','NomSituacaoI','DatSolicitacaoIV','DatPrevistaEmissaoIV','DatEmissaoIV','DatValidadeIV','NomSituacaoIV','DatSolicitacaoV','DatPrevistaEmissaoV','DatEmissaoV','DatValidadeV','NomSituacaoV','IdcTipoEntrave','DscSolucao','DscObservacao']
rename_cols = {
'DatSolicitacaoI' : 'DatSolicitacaoLP',
'DatPrevistaEmissaoI' : 'DatPrevistaEmissaoLP',
'DatEmissaoI' : 'DatEmissaoLP',
'DatValidadeI' : 'DatValidadeLP',
'NomSituacaoI' : 'NomSituacaoLP',
'DatSolicitacaoIV' : 'DatSolicitacaoLI',
'DatPrevistaEmissaoIV' : 'DatPrevistaEmissaoLI',
'DatEmissaoIV' : 'DatEmissaoLI',
'DatValidadeIV' : 'DatValidadeLI',
'NomSituacaoIV' : 'NomSituacaoLI',
'DatSolicitacaoV' : 'DatSolicitacaoLO',
'DatPrevistaEmissaoV' : 'DatPrevistaEmissaoLO',
'DatEmissaoV' : 'DatEmissaoLO',
'DatValidadeV' : 'DatValidadeLO',
'NomSituacaoV' : 'NomSituacaoLO',
'DscObservacao' : 'DscObservacaoAmbiental'}
ambientaldatas =  pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [ ]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoI','DatAssinaturaContratoI','DatValidadeContratoI','NomSitContratoII','DatAssinaturaContratoII','DatValidadeContratoII','NomSituacaoContratoIII','DatAssinaturaContratoIII','DatValidadeContratoIII','NomSitContratoIV','DatAssinaturaContratoIV','DatValidadeContratoIV','DscObservacao']
rename_cols = {'NomSitContratoI' : 'NomSitContratoCCD',
'DatAssinaturaContratoI' : 'DatAssinaturaContratoCCD',
'DatValidadeContratoI' : 'DatValidadeContratoCCD',
'NomSitContratoII' : 'NomSitContratoCCT',
'DatAssinaturaContratoII' : 'DatAssinaturaContratoCCT',
'DatValidadeContratoII' : 'DatValidadeContratoCCT',
'NomSituacaoContratoIII' : 'NomSituacaoContratoCUSD',
'DatAssinaturaContratoIII' : 'DatAssinaturaContratoCUSD',
'DatValidadeContratoIII' : 'DatValidadeContratoCUSD',
'NomSitContratoIV' : 'NomSitContratoCUST',
'DatAssinaturaContratoIV' : 'DatAssinaturaContratoCUST',
'DatValidadeContratoIV' : 'DatValidadeContratoCUST',
'DscObservacao' : 'DscObservacaoAcesso'}
acessocontratos =  pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [ ]:
cols1 = ['IdeUsinaOutorga','DthEnvio','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','DscObservacaomercado']
cols2 = ['IdeUsinaOutorga','DthEnvio','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAmbiental']
cols3 = ["IdeUsinaOutorga","DthEnvio",'IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoCCD','DatAssinaturaContratoCCD','DatValidadeContratoCCD','NomSitContratoCCT','DatAssinaturaContratoCCT','DatValidadeContratoCCT','NomSituacaoContratoCUSD','DatAssinaturaContratoCUSD','DatValidadeContratoCUSD','NomSitContratoCUST','DatAssinaturaContratoCUST','DatValidadeContratoCUST','DscObservacaoAcesso', ]
inputvrapeelv = pd.merge(mercado[cols1],ambientaldatas[cols2],on=["IdeUsinaOutorga","DthEnvio"],how="inner")
inputvrapeelv = pd.merge(inputvrapeelv,acessocontratos[cols3],on=["IdeUsinaOutorga","DthEnvio"],how="inner")

In [ ]:
vrapeelatualizado = inputvrapeelv.loc[:,['IdeUsinaOutorga','DthEnvio']]
vrapeelatualizado =vrapeelatualizado.loc[vrapeelatualizado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()].rename(columns={'DthEnvio': 'DthEnvioMAX'})

In [ ]:
cols1 = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoCCD','DatAssinaturaContratoCCD','DatValidadeContratoCCD','NomSitContratoCCT','DatAssinaturaContratoCCT','DatValidadeContratoCCT','NomSituacaoContratoCUSD','DatAssinaturaContratoCUSD','DatValidadeContratoCUSD','NomSitContratoCUST','DatAssinaturaContratoCUST','DatValidadeContratoCUST','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAcesso','DscObservacaomercado','DscObservacaoAmbiental']
cols2 = ['IdeUsinaOutorga','DthEnvioMAX']
cols3 = ['IdeUsinaOutorga','DscObservacao','DthEnvio']
definicoes = pd.merge(inputvrapeelv[cols1],vrapeelatualizado[cols2],left_on=['IdeUsinaOutorga','DthEnvio'],right_on=['IdeUsinaOutorga','DthEnvioMAX'],how="inner")
definicoes = pd.merge(definicoes,ugrapeel[cols3],on=['IdeUsinaOutorga','DthEnvio'],how="inner").drop_duplicates().rename(columns={
    'DscObservacao' : 'DscObservacaoUG'
})

dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]

In [ ]:
definicoesmonitoramento = pd.merge(monitoramentodados,definicoes,on="IdeUsinaOutorga",how="left")

definicoesmonitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoesmonitoramento.loc[definicoesmonitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]
definicoesmonitoramento.loc[definicoesmonitoramento.CondicaoAmbiental.isna(),'CondicaoAmbiental'] = 'Sem LP'
definicoesmonitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental,CondicaoAmbiental
2890,NaT,NaT,2021-11-19,2021-11-19,LP
1620,NaT,2023-02-08,2021-07-17,2023-02-08,LI
307,NaT,2021-01-24,2019-05-15,2021-01-24,LI
549,NaT,2023-11-07,2021-10-21,2023-11-07,LI
163,2026-11-10,2023-01-14,2013-01-18,2026-11-10,LO
913,NaT,2023-12-04,2019-08-23,2023-12-04,LI
553,NaT,2026-09-21,2023-10-25,2026-09-21,LI
224,2026-10-25,2026-12-06,2020-06-01,2026-10-25,LO
601,NaT,2022-03-08,2017-05-06,2022-03-08,LI
673,NaT,NaT,2021-09-16,2021-09-16,LP


In [ ]:
definicoesmonitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSitContratoCCD ==  nsa)&
    (definicoesmonitoramento.NomSitContratoCCT == nsa) &
    definicoesmonitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoesmonitoramento.loc[
(    ((definicoesmonitoramento.NomSitContratoCCD ==  valido) |
    (definicoesmonitoramento.NomSitContratoCCT == valido)) &
    definicoesmonitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.NomSitContratoCCD ==  vencido) |
    (definicoesmonitoramento.NomSitContratoCCT == vencido) |
    (definicoesmonitoramento.NomSitContratoCCD == nass) |
    (definicoesmonitoramento.NomSitContratoCCT == nass )) &
    definicoesmonitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoesmonitoramento[['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoesmonitoramento['CondicaoUso'] = "Não informado"

definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoesmonitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoesmonitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoesmonitoramento.NomSitContratoCUST == vencido) |
    (definicoesmonitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoesmonitoramento.NomSitContratoCUST == nass )) &
    definicoesmonitoramento.DthEnvio.notna())
    ,'CondicaoUso'] = "Sem Uso"

display(definicoesmonitoramento[['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoesmonitoramento["PPA"] = "Ambos"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoesmonitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoesmonitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoesmonitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoesmonitoramento[['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
32,Não se Aplica,Não se Aplica,2022-12-01 18:44:17.470,Verificar
485,Não se Aplica,Não Assinado,2022-12-02 13:38:07.643,Sem Conexão
2755,Não se Aplica,Não Assinado,2022-12-03 20:38:17.227,Sem Conexão
82,Não se Aplica,Válido,2022-12-05 15:06:43.773,OK
2514,NaN,NaN,NaT,Não informado
1154,Não se Aplica,Válido,2022-12-05 10:58:24.130,OK
1078,Não se Aplica,Válido,2021-03-05 17:34:54.263,OK
2089,Não se Aplica,Válido,2022-12-05 16:23:56.337,OK
2515,NaN,NaN,NaT,Não informado
1271,Não se Aplica,Não Assinado,2022-09-01 16:56:36.013,Sem Conexão


,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1525,Não se Aplica,Válido,Sem Conexão
275,Não se Aplica,Válido,OK
2679,Não se Aplica,Não Assinado,Sem Conexão
2221,Não se Aplica,Válido,OK
714,Não se Aplica,Válido,OK
2324,Não se Aplica,Não Assinado,Sem Conexão
288,NaN,NaN,Não informado
2075,Não se Aplica,Válido,OK
2876,Não se Aplica,Não se Aplica,Verificar
833,Não se Aplica,Válido,OK


,DscComercializacaoEnergia,DatConclusaoACL,PPA
1326,Fora do ACR,NaT,Nenhum
1376,Fora do ACR,NaT,Nenhum
616,Fora do ACR,NaT,Nenhum
1003,ACR,2022-01-25,Ambos
1633,Fora do ACR,NaT,Nenhum
2500,Fora do ACR,NaT,Nenhum
641,ACR,NaT,ACR
1713,Fora do ACR,NaT,Nenhum
903,ACR,NaT,ACR
1605,Fora do ACR,NaT,Nenhum


In [ ]:
tabelcriterio = definicoesmonitoramento.loc[definicoesmonitoramento.IdcUsinaMonitorada == "Sim"].copy()

In [ ]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabelcriterio['criterio'] = np.select(
    [
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabelcriterio['dscriterio'] = np.select(
    [
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabelcriterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
1978,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1135,Em andamento,ACR,LO,OK,0,Usina em Obras em andamento
1093,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2201,Não Iniciada,Nenhum,Sem LP,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1062,Não Iniciada,Nenhum,Sem LP,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1067,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"
393,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
2397,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"
1998,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1831,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"


In [ ]:
slp = "Sem LP"
sim = "Sim"
tabelcriterio['criterio_novo'] = np.select(
    [
        (tabelcriterio.IdcSemPrevisao == sim),
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        (tabelcriterio.CondicaoAmbiental == "Sem LP"),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabelcriterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
1596,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1446,Em andamento,Ambos,LI,OK,0,Usina em Obras em andamento
2813,Não Iniciada,Nenhum,Sem LP,Verificar,9,Sem LP
1884,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2132,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2246,Não Iniciada,ACL,LP,Sem Uso,1,Sem obras e PPA Ok
2719,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2443,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2273,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1352,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [ ]:
direcory_sas_files = r"C:\Users\viniciusbarros\OneDrive - unb.br\0Aneel\Previsor"

In [ ]:
cols1 = ['Data_Envio','IdeUsinaOutorga','CodCeg','usina_nome','canteiroReal','prev_IO','IO_real','DesvRio_real','Ench_Real','IOTrans_Real','Conc_Trans_REAL','Comiss_Real','ME_Real_conc_eol','Conc_Combust_Real',]
vinputcronograma = vcronograma[cols].copy()

KeyError: "['NomUsina', 'DatRealizacaoII', 'DatPrevistaAprovacaoIII', 'DatRealizacaoIII', 'DatRealizacaoIX', 'DatRealizacaoXI', 'DatRealizacaoXIII', 'DatRealizacaoXIV', 'DatRealizacaoXII', 'DatRealizacaoVII', 'DatRealizacaoVIII'] not in index"

In [ ]:
vcronograma.columns

Index(['IdeUsinaOutorga', 'CodCeg', 'usina_nome', 'canteiroReal', 'prev_IO',
       'IO_real', 'DesvRio_real', 'Ench_Real', 'IOTrans_Real',
       'Conc_Trans_REAL', 'Comiss_Real', 'ME_Real_conc_eol',
       'Conc_Combust_Real', 'DatRealizacaoVI', 'DatRealizacaoX',
       'DatRealizacaoV', 'DatRealizacaoIV'],
      dtype='object')

In [ ]:
vcronograma[cols1]

KeyError: "['Data_Envio'] not in index"